In [1]:
DEBUG = False

In [2]:
import numpy as np
import random
import pandas as pd

In [3]:
from collections import defaultdict
import datatable as dt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import gc

_ = np.seterr(divide='ignore', invalid='ignore')

In [4]:
data_types_dict = {
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id':'int8', 
    'task_container_id': 'int16',
    #'user_answer': 'int8',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'

In [5]:
print('start read train data...')
train_df = dt.fread('./input/train.csv', columns=set(data_types_dict.keys())).to_pandas()

start read train data...


In [6]:
len(train_df)

101230332

In [7]:
part_df = pd.read_pickle('train_df_part.pkl')
train_df = pd.concat([train_df, part_df], axis=1)



# fe

In [8]:
# 同じ問題(content_id)に取り組んだlagtime
train_df['content_lagtime'] = train_df.groupby(['user_id', 'content_id'])['timestamp'].shift()

max_timestamp_u_content = train_df[['user_id', 'content_id','timestamp']].groupby(['user_id', 'content_id']).agg(['max']).reset_index()
max_timestamp_u_content.columns = ['user_id', 'content_id', 'timestamp']
max_timestamp_u_content.user_id=max_timestamp_u_content.user_id.astype('int32')
max_timestamp_u_content.to_pickle('max_timestamp_u_content_org.pkl')
max_timestamp_u_content=max_timestamp_u_content[max_timestamp_u_content['timestamp'] >43712886024] # timestamp.max(87425772049) / 2
# max_timestamp_u_content.to_pickle('max_timestamp_u_content.pkl')
del max_timestamp_u_content

In [9]:
train_df['content_lagtime']=train_df['timestamp']-train_df['content_lagtime']
train_df['content_lagtime'].fillna(-1, inplace=True)

train_df['content_lagtime']=train_df['content_lagtime']/(1000*3600)
train_df.lagtime=train_df.content_lagtime.astype('float32')

/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [10]:
gc.collect()

46

In [11]:

#1= if the event was the user watching a lecture.
cum = train_df.groupby('user_id')['content_type_id'].agg(['cumsum', 'cumcount'])
cum['cumcount']=cum['cumcount']+1
train_df['user_interaction_count'] = cum['cumcount']
# train_df['user_interaction_timestamp_mean'] = train_df['timestamp']/cum['cumcount']
# train_df['user_lecture_sum'] = cum['cumsum']
# train_df['user_lecture_lv'] = cum['cumsum'] / cum['cumcount']


# train_df.user_lecture_lv=train_df.user_lecture_lv.astype('float16')
# train_df.user_lecture_sum=train_df.user_lecture_sum.astype('int16')
train_df.user_interaction_count=train_df.user_interaction_count.astype('int16')
# train_df['user_interaction_timestamp_mean']=train_df['user_interaction_timestamp_mean']/(1000*3600)
# train_df.user_interaction_timestamp_mean=train_df.user_interaction_timestamp_mean.astype('float32')


In [12]:
del cum
gc.collect()

53

In [13]:
print('start handle train_df...')

start handle train_df...


In [14]:
train_df['lagtime_in_lecture'] = train_df.groupby('user_id')['timestamp'].shift()

train_df['lagtime_in_lecture']=train_df['timestamp']-train_df['lagtime_in_lecture']
lagtime_in_lecture_mean=train_df['lagtime_in_lecture'].mean()
train_df['lagtime_in_lecture'].fillna(lagtime_in_lecture_mean, inplace=True)

train_df['lagtime_in_lecture']=train_df['lagtime_in_lecture']/(1000*3600)
train_df.lagtime_in_lecture=train_df.lagtime_in_lecture.astype('float32')

In [15]:
# add
# train_df['lag'] = train_df.groupby(['user_id', 'part'])[target].shift()

cum = train_df.groupby(['user_id', 'part'])['content_type_id'].agg(['cumsum', 'cumcount'])
##cum['cumcount']=cum['cumcount']+1
part_lecture_user_agg = train_df.groupby(['user_id', 'part'])['content_type_id'].agg(['sum', 'count']).astype('int16')
part_lecture_user_agg.to_pickle('./intermed/part_lecture_user_agg.pkl')
del part_lecture_user_agg

cum['cumsum'].fillna(0, inplace=True)
train_df['user_lecture_sum'] = cum['cumsum']
train_df['user_lecture_lv'] = cum['cumsum'] / cum['cumcount']
#train_df['user_answer_count'] = cum['cumcount']

# train_df.drop(columns=['lag'], inplace=True)
train_df.user_lecture_lv=train_df.user_lecture_lv.astype('float16')
train_df.user_lecture_sum=train_df.user_lecture_sum.astype('int16')



In [16]:
train_df['prior_question_had_explanation'].fillna(False, inplace=True)
train_df = train_df.astype(data_types_dict)
train_df = train_df[train_df[target] != -1].reset_index(drop=True) # lecture削除

In [17]:
content_explation_agg=train_df[["content_id","prior_question_had_explanation",target]].groupby(["content_id","prior_question_had_explanation"])[target].agg(['mean'])

In [18]:
content_explation_agg=content_explation_agg.unstack()

content_explation_agg=content_explation_agg.reset_index()
content_explation_agg.columns = ['content_id', 'content_explation_false_mean','content_explation_true_mean']

In [19]:
content_explation_agg.content_id=content_explation_agg.content_id.astype('int16')
content_explation_agg.content_explation_false_mean=content_explation_agg.content_explation_false_mean.astype('float16')
content_explation_agg.content_explation_true_mean=content_explation_agg.content_explation_true_mean.astype('float16')

In [20]:
print('start handle attempt_no...')

start handle attempt_no...


In [21]:

train_df["attempt_no"] = 1
train_df.attempt_no=train_df.attempt_no.astype('int8')
#
attempt_no_agg=train_df.groupby(["user_id","content_id"])["attempt_no"].agg(['sum']).astype('int8')
#attempt_no_agg=attempt_no_agg.astype('int8')
train_df["attempt_no"] = train_df[["user_id","content_id",'attempt_no']].groupby(["user_id","content_id"])["attempt_no"].cumsum()

In [22]:
#attempt_no_agg=attempt_no_agg.reset_index()
attempt_no_agg=attempt_no_agg[attempt_no_agg['sum'] >1]

# attempt_no_agg.to_pickle('attempt_no_agg.pkl')
del attempt_no_agg

In [23]:
cum = train_df.groupby('user_id')['attempt_no'].agg(['cumsum', 'cumcount'])
##cum['cumcount']=cum['cumcount']+1
attempt_cum_agg = train_df.groupby('user_id')['attempt_no'].agg(['sum', 'count']).astype('int16')
# attempt_cum_agg.to_pickle('attempt_cum_agg.pkl')
del attempt_cum_agg

cum['cumsum'].fillna(0, inplace=True)

train_df['attempt_no_mean'] = cum['cumsum'] / cum['cumcount']
train_df.attempt_no_mean=train_df.attempt_no_mean.astype('float16')

In [24]:
print('start handle timestamp...')
prior_question_elapsed_time_mean=train_df['prior_question_elapsed_time'].mean()
train_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace=True)

start handle timestamp...


In [25]:

train_df['lag'] = train_df.groupby('user_id')[target].shift()

cum = train_df.groupby('user_id')['lag'].agg(['cumsum', 'cumcount'])
##cum['cumcount']=cum['cumcount']+1
user_agg = train_df.groupby('user_id')['lag'].agg(['sum', 'count']).astype('int16')
cum['cumsum'].fillna(0, inplace=True)

train_df['user_correctness'] = cum['cumsum'] / cum['cumcount']
train_df['user_cumcount'] = cum['cumcount']
train_df['user_correct_count'] = cum['cumsum']
train_df['user_uncorrect_count'] = cum['cumcount']-cum['cumsum']
#train_df['user_answer_count'] = cum['cumcount']
train_df.drop(columns=['lag'], inplace=True)
train_df['user_correctness'].fillna(0.67, inplace=True)
train_df.user_correctness=train_df.user_correctness.astype('float16')
train_df.user_correct_count=train_df.user_correct_count.astype('int16')
train_df.user_uncorrect_count=train_df.user_uncorrect_count.astype('int16')
#train_df.user_answer_count=train_df.user_answer_count.astype('int16')


In [26]:
# add
train_df['lag'] = train_df.groupby(['user_id', 'part'])[target].shift()

cum = train_df.groupby(['user_id', 'part'])['lag'].agg(['cumsum', 'cumcount'])
##cum['cumcount']=cum['cumcount']+1
part_user_agg = train_df.groupby(['user_id', 'part'])['lag'].agg(['sum', 'count']).astype('int16')
# part_user_agg.to_pickle('./intermed/part_user_agg.pkl')
del part_user_agg

cum['cumsum'].fillna(0, inplace=True)

train_df['user_part_correctness'] = cum['cumsum'] / cum['cumcount'] # part別正答率
# train_df['user_part_rate'] = cum['cumcount'] / train_df['user_cumcount']# part別回答割合
#train_df['user_answer_count'] = cum['cumcount']
train_df.drop(columns=['lag', 'user_cumcount'], inplace=True)
train_df['user_part_correctness'].fillna(0.67, inplace=True)
train_df.user_part_correctness=train_df.user_part_correctness.astype('float16')
# train_df.user_part_rate=train_df.user_part_rate.astype('float16')


In [27]:
# add
# 同じ問partに取り組んだlagtime
train_df['part_lagtime'] = train_df.groupby(['user_id', 'part'])['timestamp'].shift()

max_timestamp_u_part = train_df[['user_id', 'part','timestamp']].groupby(['user_id', 'part']).agg(['max']).reset_index()
max_timestamp_u_part.columns = ['user_id', 'part', 'timestamp']
max_timestamp_u_part.user_id=max_timestamp_u_part.user_id.astype('int32')
max_timestamp_u_part.to_pickle('max_timestamp_u_part_org.pkl')
max_timestamp_u_part=max_timestamp_u_part[max_timestamp_u_part['timestamp'] >43712886024] # timestamp.max(87425772049) / 2
# max_timestamp_u_part.to_pickle('max_timestamp_u_part.pkl')
del max_timestamp_u_part

train_df['part_lagtime']=train_df['timestamp']-train_df['part_lagtime']
train_df['part_lagtime'].fillna(-1, inplace=True)

train_df['part_lagtime']=train_df['part_lagtime']/(1000*3600)
train_df.part_lagtime=train_df.part_lagtime.astype('float32')

train_df.drop(columns=['part'], inplace=True)

In [28]:
max_timestamp_u = train_df[['user_id','timestamp']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u.columns = ['user_id', 'max_time_stamp']
max_timestamp_u.user_id=max_timestamp_u.user_id.astype('int32')
# max_timestamp_u.to_pickle('max_timestamp_u.pkl')

del max_timestamp_u

In [29]:

train_df['lagtime'] = train_df.groupby('user_id')['timestamp'].shift()

max_timestamp_u2 = train_df[['user_id','lagtime']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u2.columns = ['user_id', 'max_time_stamp2']
max_timestamp_u2.user_id=max_timestamp_u2.user_id.astype('int32')

# max_timestamp_u2.to_pickle('max_timestamp_u2.pkl')
del max_timestamp_u2

train_df['lagtime']=train_df['timestamp']-train_df['lagtime']
lagtime_mean=train_df['lagtime'].mean()
train_df['lagtime'].fillna(lagtime_mean, inplace=True)


In [30]:
train_df['lag_elapsed_time'] = train_df['lagtime'] - train_df['prior_question_elapsed_time']

lag_elapsed_time_mean=train_df['lag_elapsed_time'].mean()
train_df['lag_elapsed_time'].fillna(lag_elapsed_time_mean, inplace=True)

In [31]:
train_df['lagtime']=train_df['lagtime']/(1000*3600)
train_df.lagtime=train_df.lagtime.astype('float32')

In [32]:

train_df['lagtime2'] = train_df.groupby('user_id')['timestamp'].shift(2)

max_timestamp_u3 = train_df[['user_id','lagtime2']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u3.columns = ['user_id', 'max_time_stamp3']
max_timestamp_u3.user_id=max_timestamp_u3.user_id.astype('int32')

# max_timestamp_u3.to_pickle('max_timestamp_u3.pkl')
del max_timestamp_u3

train_df['lagtime2']=train_df['timestamp']-train_df['lagtime2']
lagtime_mean2=train_df['lagtime2'].mean()
train_df['lagtime2'].fillna(lagtime_mean2, inplace=True)


In [33]:
train_df['lagtime2']=train_df['lagtime2']/(1000*3600)
train_df.lagtime2=train_df.lagtime2.astype('float32')

In [34]:

train_df['lagtime3'] = train_df.groupby('user_id')['timestamp'].shift(3)

train_df['lagtime3']=train_df['timestamp']-train_df['lagtime3']
lagtime_mean3=train_df['lagtime3'].mean()
train_df['lagtime3'].fillna(lagtime_mean3, inplace=True)
train_df['lagtime3']=train_df['lagtime3']/(1000*3600)
train_df.lagtime3=train_df.lagtime3.astype('float32')

In [35]:
train_df['lagtime1_2'] = train_df['lagtime'] - train_df['lagtime2']
train_df['lagtime2_3'] = train_df['lagtime2'] - train_df['lagtime3']

In [36]:
train_df['timestamp']=train_df['timestamp']/(1000*3600)
#
train_df.timestamp=train_df.timestamp.astype('float16')

In [37]:
user_prior_question_elapsed_time = train_df[['user_id','prior_question_elapsed_time']].groupby(['user_id']).tail(1)
user_prior_question_elapsed_time.columns = ['user_id', 'prior_question_elapsed_time']

# user_prior_question_elapsed_time.to_pickle('user_prior_question_elapsed_time.pkl')
del user_prior_question_elapsed_time

In [38]:
del cum
gc.collect()

133

In [39]:
train_df.prior_question_had_explanation=train_df.prior_question_had_explanation.astype('int8')
explanation_agg = train_df.groupby('user_id')['prior_question_had_explanation'].agg(['sum', 'count'])
explanation_agg=explanation_agg.astype('int16')
# explanation_agg.sum=explanation_agg.sum.astype('int16')
# explanation_agg.count=explanation_agg.count.astype('int16')
#explanation_agg.var=explanation_agg.var.astype('float16')

# explanation_agg.to_pickle('explanation_agg.pkl')
del explanation_agg

In [40]:


#train_df['lag'] = train_df.groupby('user_id')['prior_question_had_explanation'].shift()

cum = train_df.groupby('user_id')['prior_question_had_explanation'].agg(['cumsum', 'cumcount'])
cum['cumcount']=cum['cumcount']+1
# train_df['explanation_mean'] = cum['cumsum'] / cum['cumcount']
# train_df['explanation_true_count'] = cum['cumsum']
train_df['explanation_false_count'] =  cum['cumcount']-cum['cumsum']
#train_df.drop(columns=['lag'], inplace=True)

# train_df.explanation_mean=train_df.explanation_mean.astype('float16')
# train_df.explanation_true_count=train_df.explanation_true_count.astype('int16')
train_df.explanation_false_count=train_df.explanation_false_count.astype('int16')

In [41]:
del cum
gc.collect()

73

In [42]:
content_agg = train_df.groupby('content_id')[target].agg(['sum', 'count','var'])
task_container_agg = train_df.groupby('task_container_id')[target].agg(['sum', 'count','var'])
content_agg=content_agg.astype('float32')
task_container_agg=task_container_agg.astype('float32')

In [43]:
len(train_df)

99271300

In [44]:
#
train_df['task_container_uncor_count'] = train_df['task_container_id'].map(task_container_agg['count']-task_container_agg['sum']).astype('int32')
# train_df['task_container_cor_count'] = train_df['task_container_id'].map(task_container_agg['sum']).astype('int32')
# train_df['task_container_std'] = train_df['task_container_id'].map(task_container_agg['var']).astype('float16')
# train_df['task_container_correctness'] = train_df['task_container_id'].map(task_container_agg['sum'] / task_container_agg['count'])
# train_df.task_container_correctness=train_df.task_container_correctness.astype('float16')

# task_container_agg.to_pickle('task_container_agg.pkl')
del task_container_agg

In [45]:
content_elapsed_time_agg=train_df.groupby('content_id')['prior_question_elapsed_time'].agg(['mean'])
content_had_explanation_agg=train_df.groupby('content_id')['prior_question_had_explanation'].agg(['mean'])

In [46]:
train_df.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,content_lagtime,user_interaction_count,...,user_part_correctness,part_lagtime,lagtime,lag_elapsed_time,lagtime2,lagtime3,lagtime1_2,lagtime2_3,explanation_false_count,task_container_uncor_count
0,0.000000,115,5692,0,1,1,13005.081055,0,-2.777778e-07,1,...,0.669922,-2.777778e-07,5.678317,2.042894e+07,11.214070,16.669001,-5.535753,-5.454930,1,187175
1,0.015823,115,5716,0,2,1,37000.000000,0,-2.777778e-07,2,...,1.000000,1.581750e-02,0.015818,1.994300e+04,11.214070,16.669001,-11.198253,-5.454930,2,223198
2,0.032867,115,128,0,0,1,55000.000000,0,-2.777778e-07,3,...,0.669922,-2.777778e-07,0.017061,6.420000e+03,0.032879,16.669001,-0.015818,-16.636122,3,126682
3,0.036438,115,7860,0,3,1,19000.000000,0,-2.777778e-07,4,...,1.000000,3.556667e-03,0.003557,-6.196000e+03,0.020618,0.036435,-0.017061,-0.015817,4,180574
4,0.038330,115,7922,0,4,1,11000.000000,0,-2.777778e-07,5,...,1.000000,1.888333e-03,0.001888,-4.202000e+03,0.005445,0.022506,-0.003557,-0.017061,5,361957


In [47]:
print('start questions data...')

start questions data...


In [48]:
questions_df = pd.read_csv(
    './input/questions.csv', 
    usecols=[0, 1,3,4],
    dtype={'question_id': 'int16','bundle_id': 'int16', 'part': 'int8','tags': 'str'}
)

In [49]:
bundle_agg = questions_df.groupby('bundle_id')['question_id'].agg(['count'])

In [50]:
questions_df['content_sub_bundle'] = questions_df['bundle_id'].map(bundle_agg['count']).astype('int8')

In [51]:

questions_df['tags'].fillna('188', inplace=True)

In [52]:
def gettags(tags,num):
    tags_splits=tags.split(" ")
    result='' 
    for t in tags_splits:
        x=int(t)
        if(x<32*(num+1) and x>=32*num):#num 
            result=result+' '+t
    return result

In [53]:
from sklearn.preprocessing import LabelEncoder
for num in range(0,6):
    questions_df["tags"+str(num)] = questions_df["tags"].apply(lambda row: gettags(row,num))
    le = LabelEncoder()
    le.fit(np.unique(questions_df['tags'+str(num)].values))
    #questions_df[['tags'+str(num)]=
    questions_df['tags'+str(num)]=questions_df[['tags'+str(num)]].apply(le.transform)

In [54]:
questions_df_dict = {   
    'tags0': 'int8',
    'tags1': 'int8',
    'tags2': 'int8',
    'tags3': 'int8',
    'tags4': 'int8',
    'tags5': 'int8',
    #'tags6': 'int8',
    #'tags7': 'int8'
}
questions_df = questions_df.astype(questions_df_dict)

In [55]:
questions_df.drop(columns=['tags'], inplace=True)

In [56]:

questions_df['part_bundle_id']=questions_df['part']*100000+questions_df['bundle_id']
questions_df.part_bundle_id=questions_df.part_bundle_id.astype('int32')
# tag = questions_df["tags"].str.split(" ", n = 10, expand = True)
# tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']
# #

# tag.fillna(0, inplace=True)
# tag = tag.astype('int16')
# questions_df =  pd.concat([questions_df,tag],axis=1).drop(['tags'],axis=1)

In [57]:
questions_df.rename(columns={'question_id':'content_id'}, inplace=True)

In [58]:
questions_df = pd.merge(questions_df, content_explation_agg, on='content_id', how='left',right_index=True)#
# questions_df.content_explation_false_mean=questions_df.content_explation_false_mean.astype('float16')
# questions_df.content_explation_true_mean=questions_df.content_explation_true_mean.astype('float16')

In [59]:
del content_explation_agg

In [60]:
questions_df['sum'] = questions_df['content_id'].map(content_agg['sum'])

In [61]:
questions_df['content_correctness'] = questions_df['content_id'].map(content_agg['sum'] / content_agg['count'])
questions_df.content_correctness=questions_df.content_correctness.astype('float16')
questions_df['content_correctness_std'] = questions_df['content_id'].map(content_agg['var'])
questions_df.content_correctness_std=questions_df.content_correctness_std.astype('float16')
questions_df['content_uncorrect_count'] = questions_df['content_id'].map(content_agg['count']-content_agg['sum']).astype('int32')
questions_df['content_correct_count'] = questions_df['content_id'].map(content_agg['sum']).astype('int32')

In [62]:
questions_df['content_elapsed_time_mean'] = questions_df['content_id'].map(content_elapsed_time_agg['mean'])
questions_df.content_elapsed_time_mean=questions_df.content_elapsed_time_mean.astype('float16')
questions_df['content_had_explanation_mean'] = questions_df['content_id'].map(content_had_explanation_agg['mean'])
questions_df.content_had_explanation_mean=questions_df.content_had_explanation_mean.astype('float16')

In [63]:
del content_elapsed_time_agg
del content_had_explanation_agg
gc.collect()

53

In [64]:
part_agg = questions_df.groupby('part')['content_correctness'].agg(['mean', 'var'])
questions_df['part_correctness_mean'] = questions_df['part'].map(part_agg['mean'])
questions_df['part_correctness_std'] = questions_df['part'].map(part_agg['var'])
questions_df.part_correctness_mean=questions_df.part_correctness_mean.astype('float16')
questions_df.part_correctness_std=questions_df.part_correctness_std.astype('float16')

In [65]:
part_agg = questions_df.groupby('part')['content_uncorrect_count'].agg(['sum'])
questions_df['part_uncor_count'] = questions_df['part'].map(part_agg['sum']).astype('int32')
#
part_agg = questions_df.groupby('part')['content_correct_count'].agg(['sum'])
questions_df['part_cor_count'] = questions_df['part'].map(part_agg['sum']).astype('int32')

In [66]:
bundle_agg = questions_df.groupby('bundle_id')['content_correctness'].agg(['mean'])
questions_df['bundle_correctness_mean'] = questions_df['bundle_id'].map(bundle_agg['mean'])
questions_df.bundle_correctness_mean=questions_df.bundle_correctness_mean.astype('float16')

# bundle_agg = questions_df.groupby('bundle_id')['content_uncorrect_count'].agg(['sum'])
# questions_df['bundle_uncor_count'] = questions_df['bundle_id'].map(bundle_agg['sum']).astype('int32')
# #
# bundle_agg = questions_df.groupby('bundle_id')['content_correct_count'].agg(['sum'])
# questions_df['bundle_cor_count'] = questions_df['bundle_id'].map(bundle_agg['sum']).astype('int32')

In [67]:
del content_agg
del bundle_agg
del part_agg
#del tags1_agg
gc.collect()

80

# train

In [68]:
# origin
features_dict = {
    #'user_id',
    'timestamp':'float16',#
    'user_interaction_count':'int16',
    'user_interaction_timestamp_mean':'float32',
    'lagtime':'float32',#
    'lagtime2':'float32',
    'lagtime3':'float32',
    #'lagtime_mean':'int32',
    'content_id':'int16',
    'task_container_id':'int16',
    'user_lecture_sum':'int16',#
    'user_lecture_lv':'float16',##
    'prior_question_elapsed_time':'float32',#
    'delta_prior_question_elapsed_time':'int32',#
    'user_correctness':'float16',#
    'user_uncorrect_count':'int16',#
    'user_correct_count':'int16',#
    #'content_correctness':'float16',
    'content_correctness_std':'float16',
    'content_correct_count':'int32',
    'content_uncorrect_count':'int32',#
    'content_elapsed_time_mean':'float16',
    'content_had_explanation_mean':'float16',
    'content_explation_false_mean':'float16',
    'content_explation_true_mean':'float16',
    'task_container_correctness':'float16',
    'task_container_std':'float16',
    'task_container_cor_count':'int32',#
    'task_container_uncor_count':'int32',#
    'attempt_no':'int8',#
    'part':'int8',
    'part_correctness_mean':'float16',
    'part_correctness_std':'float16',
    'part_uncor_count':'int32',
    'part_cor_count':'int32',
    'tags0': 'int8',
    'tags1': 'int8',
    'tags2': 'int8',
    'tags3': 'int8',
    'tags4': 'int8',
    'tags5': 'int8',
   # 'tags6': 'int8',
   # 'tags7': 'int8',
#     'tags0_correctness_mean':'float16',
#     'tags1_correctness_mean':'float16',
#     'tags2_correctness_mean':'float16',
#     'tags4_correctness_mean':'float16',
#     'bundle_id':'int16',
#     'bundle_correctness_mean':'float16',
#     'bundle_uncor_count':'int32',
#     'bundle_cor_count':'int32',
    'part_bundle_id':'int32',
    'content_sub_bundle':'int8',
    'prior_question_had_explanation':'int8',
    'explanation_mean':'float16', #
    #'explanation_var',#
    'explanation_false_count':'int16',#
    'explanation_true_count':'int16',#
   # 'community':'int8',
#     'part_1',
#     'part_2',
#     'part_3',
#     'part_4',
#     'part_5',
#     'part_6',
#     'part_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
}
categorical_columns= [
    #'user_id',
    'content_id',
    'task_container_id',
    'part',
   # 'community',
    'tags0',
    'tags1',
    'tags2',
    'tags3',
    'tags4',
    'tags5',
    #'tags6',
    #'tags7',
    #'bundle_id',
    'part_bundle_id',
    'content_sub_bundle',
    'prior_question_had_explanation', 
#     'part_1',
#     'part_2',
#     'part_3',
#     'part_4',
#     'part_5',
#     'part_6',
#     'part_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
]

features=list(features_dict.keys())


In [69]:
###### Value>100000
features_dict = {
#  'lagtime': 'float32', # curr
 'lagtime2': 'float32', #
 'lagtime3': 'float32',
 'content_id': 'int16',
#  'task_container_id': 'int16', # curr
 'prior_question_elapsed_time': 'float32',
 'user_correctness': 'float16',
#  'user_uncorrect_count': 'int16', # curr
 'user_correct_count': 'int16',
 'content_correctness_std': 'float16',
 'content_correct_count': 'int32',
 'content_uncorrect_count': 'int32',
 'content_had_explanation_mean': 'float16', #
#  'content_explation_false_mean': 'float16', # curr
 'content_explation_true_mean': 'float16',
 'attempt_no': 'int8',
#  'part': 'int8', # curr
 'part_correctness_mean': 'float16',
 'part_bundle_id': 'int32',
 'explanation_false_count': 'int16',
 'lagtime_in_lecture': 'float32',
 'content_lagtime': 'float32',
#  'lag_elapsed_time': 'float32', # curr
 'attempt_no_mean': 'float16',
 'lagtime1_2': 'float32',
 'lagtime2_3': 'float32',
 'user_part_correctness': 'float16',
#  'user_part_rate': 'float16',
 'part_lagtime': 'float16',
 'user_lecture_sum': 'float16',
 'user_lecture_lv': 'int16',
#  'community': 'int16',
#  'community_correctness_mean': 'float16',
#  'lecture_of_day': 'int16',
#  'user_interaction_lagtime_mean': 'float32',
#  'lagtime_cumsum': 'int16'
}

features=list(features_dict.keys())

delete_col = set(categorical_columns) - set(features)
categorical_columns = list(set(categorical_columns) - delete_col)
print(categorical_columns)

['part_bundle_id', 'content_id']


In [70]:
flag_lgbm=True
clfs = list()
params = {
'num_leaves': 300,
'max_bin':450,
# 'min_child_weight': 0.03454472573214212,
'feature_fraction': 0.52,
'bagging_fraction': 0.52,
#'min_data_in_leaf': 106,
# 'max_depth': -1,
'objective': 'binary',
'learning_rate': 0.05,
"boosting_type": "gbdt",
"metric": 'auc',
# "bagging_seed": 11,
# "verbosity": -1,
# 'reg_alpha': 0.3899927210061127,
# 'reg_lambda': 0.6485237330340494,
# 'random_state': 47
}
trains=list()
valids=list()
num=1
for i in range(0,num):
    
    #train_df=train_df.reset_index(drop=True)
    #train_df_clf=train_df.sample(n=1200*10000)
    if DEBUG:
        train_df_clf=train_df[:10000]
    else:
#         train_df_clf=train_df[1200*10000:int(2.5*1200)*10000]
        train_df_clf=train_df.copy()
    
    print('sample end')
    #train_df.drop(train_df_clf.index, inplace=True)
    #print('train_df drop end')
    
   
    del train_df
    
    
    
    users=train_df_clf['user_id'].drop_duplicates()#
    #
    users=users.sample(frac=0.08)
    users_df=pd.DataFrame()
    users_df['user_id']=users.values
   
   
    valid_df_newuser = pd.merge(train_df_clf, users_df, on=['user_id'], how='inner',right_index=True)
    del users_df
    del users
    gc.collect()
    #
    train_df_clf.drop(valid_df_newuser.index, inplace=True)
    print('pd.merge(train_df_clf, questions_df)')
    #-----------
    #train_df_clf=train_df_clf.sample(frac=0.2)
    #train_df_clf.drop(valid_df_newuser.index, inplace=True)
    train_df_clf = pd.merge(train_df_clf, questions_df, on='content_id', how='left',right_index=True)#
    valid_df_newuser = pd.merge(valid_df_newuser, questions_df, on='content_id', how='left',right_index=True)#
    del questions_df
#     train_df_clf = pd.merge(train_df_clf, user_lecture_stats_part, on='user_id', how="left",right_index=True)
#     valid_df_newuser = pd.merge(valid_df_newuser, user_lecture_stats_part, on='user_id', how="left",right_index=True)
    print('valid_df')
    valid_df=train_df_clf.sample(frac=0.1)
    train_df_clf.drop(valid_df.index, inplace=True)
    
#     test_df=train_df_clf.sample(n=100*10000)
#     train_df_clf.drop(test_df.index, inplace=True)
   
    valid_df = valid_df.append(valid_df_newuser)
    del valid_df_newuser
    gc.collect()
    #

    trains.append(train_df_clf)
    valids.append(valid_df)
    print('train_df_clf length：',len(train_df_clf))
    print('valid_df length：',len(valid_df))
    #train_df=train_df.reset_index(drop=True)

sample end
pd.merge(train_df_clf, questions_df)
valid_df
train_df_clf length： 81973662
valid_df length： 17297638


In [71]:
#del train_df
del train_df_clf
del valid_df
gc.collect()

20

In [72]:

for i in range(0,num):
      
#     tr_data = lgb.Dataset(trains[i][features], label=trains[i][target])
#     va_data = lgb.Dataset(valids[i][features], label=valids[i][target])

    #Don't use DF to create lightgbm dataset, rather use np array:
    X_train_np = trains[i][features].values.astype(np.float32)
    X_valid_np = valids[i][features].values.astype(np.float32)
    #features = train.columns
    tr_data = lgb.Dataset(X_train_np, label=trains[i][target], feature_name=list(features))
    va_data = lgb.Dataset(X_valid_np, label=valids[i][target], feature_name=list(features))
    

#     del train_df_clf
#     del valid_df
#     gc.collect()
    del trains
    del valids
    del X_train_np
    del X_valid_np
    gc.collect()

    model = lgb.train(
        params, 
        tr_data,
#         train_df[features],
#         train_df[target],
        num_boost_round=1400,
        #valid_sets=[(train_df[features],train_df[target]), (valid_df[features],valid_df[target])], 
        valid_sets=[tr_data, va_data],
        early_stopping_rounds=20,
        feature_name=features,
        categorical_feature=categorical_columns,
        verbose_eval=1
    )
    clfs.append(model)
    print('Training done!!!')
    #print('test-auc:', roc_auc_score(test_df[target], model.predict(test_df[features])))
    #model.save_model(f'model.txt')


#     fig,ax = plt.subplots(figsize=(15,15))
#     lgb.plot_importance(model, ax=ax,importance_type='gain',max_num_features=50)
#     plt.show()

    del tr_data
    del va_data
    gc.collect()
#    
# del trains
# del valids
# gc.collect()
# del test_df
# gc.collect()

/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['content_id', 'part_bundle_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Info] Number of positive: 53868920, number of negative: 28104742
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.479528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 27265
[LightGBM] [Info] Number of data points in the train set: 81973662, number of used features: 24


/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.657149 -> initscore=0.650615
[LightGBM] [Info] Start training from score 0.650615
[1]	training's auc: 0.746369	valid_1's auc: 0.746076
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.758096	valid_1's auc: 0.757849
[3]	training's auc: 0.759978	valid_1's auc: 0.759694
[4]	training's auc: 0.763564	valid_1's auc: 0.763317
[5]	training's auc: 0.76443	valid_1's auc: 0.764116
[6]	training's auc: 0.765079	valid_1's auc: 0.764797
[7]	training's auc: 0.764628	valid_1's auc: 0.764381
[8]	training's auc: 0.766154	valid_1's auc: 0.765926
[9]	training's auc: 0.76715	valid_1's auc: 0.766914
[10]	training's auc: 0.767965	valid_1's auc: 0.767738
[11]	training's auc: 0.768231	valid_1's auc: 0.767999
[12]	training's auc: 0.768994	valid_1's auc: 0.768744
[13]	training's auc: 0.768926	valid_1's auc: 0.768659
[14]	training's auc: 0.769596	valid_1's auc: 0.769339
[15]	training's auc: 0.769683	valid_1's auc: 0.769432
[16]	tra

[150]	training's auc: 0.784739	valid_1's auc: 0.78293
[151]	training's auc: 0.7848	valid_1's auc: 0.782972
[152]	training's auc: 0.784852	valid_1's auc: 0.783024
[153]	training's auc: 0.784899	valid_1's auc: 0.78305
[154]	training's auc: 0.784927	valid_1's auc: 0.783076
[155]	training's auc: 0.784983	valid_1's auc: 0.783112
[156]	training's auc: 0.785047	valid_1's auc: 0.783153
[157]	training's auc: 0.785097	valid_1's auc: 0.78318
[158]	training's auc: 0.785172	valid_1's auc: 0.783238
[159]	training's auc: 0.785224	valid_1's auc: 0.783273
[160]	training's auc: 0.785253	valid_1's auc: 0.783302
[161]	training's auc: 0.78531	valid_1's auc: 0.783336
[162]	training's auc: 0.78536	valid_1's auc: 0.783373
[163]	training's auc: 0.785412	valid_1's auc: 0.7834
[164]	training's auc: 0.785467	valid_1's auc: 0.78343
[165]	training's auc: 0.785498	valid_1's auc: 0.783461
[166]	training's auc: 0.78557	valid_1's auc: 0.78351
[167]	training's auc: 0.785626	valid_1's auc: 0.783551
[168]	training's auc: 

...

[750]	training's auc: 0.795788	valid_1's auc: 0.788028
[751]	training's auc: 0.795795	valid_1's auc: 0.788028
[752]	training's auc: 0.795803	valid_1's auc: 0.788035
[753]	training's auc: 0.795807	valid_1's auc: 0.788036
[754]	training's auc: 0.795811	valid_1's auc: 0.788035
[755]	training's auc: 0.795824	valid_1's auc: 0.788039
[756]	training's auc: 0.795829	valid_1's auc: 0.788038
[757]	training's auc: 0.795832	valid_1's auc: 0.788038
[758]	training's auc: 0.795836	valid_1's auc: 0.78804
[759]	training's auc: 0.795844	valid_1's auc: 0.78804
[760]	training's auc: 0.795848	valid_1's auc: 0.78804
[761]	training's auc: 0.795854	valid_1's auc: 0.788041
[762]	training's auc: 0.795866	valid_1's auc: 0.788043
[763]	training's auc: 0.795867	valid_1's auc: 0.788044
[764]	training's auc: 0.795871	valid_1's auc: 0.788045
[765]	training's auc: 0.795884	valid_1's auc: 0.788051
[766]	training's auc: 0.79589	valid_1's auc: 0.788057
[767]	training's auc: 0.795895	valid_1's auc: 0.788062
[768]	training

[900]	training's auc: 0.796717	valid_1's auc: 0.788255
[901]	training's auc: 0.796719	valid_1's auc: 0.788255
[902]	training's auc: 0.796728	valid_1's auc: 0.788255
[903]	training's auc: 0.796732	valid_1's auc: 0.788255
[904]	training's auc: 0.796734	valid_1's auc: 0.788255
[905]	training's auc: 0.796736	valid_1's auc: 0.788255
[906]	training's auc: 0.796746	valid_1's auc: 0.788255
[907]	training's auc: 0.79675	valid_1's auc: 0.788258
[908]	training's auc: 0.796752	valid_1's auc: 0.788258
[909]	training's auc: 0.796759	valid_1's auc: 0.788259
[910]	training's auc: 0.796765	valid_1's auc: 0.788258
[911]	training's auc: 0.796766	valid_1's auc: 0.788258
[912]	training's auc: 0.796768	valid_1's auc: 0.788259
[913]	training's auc: 0.796773	valid_1's auc: 0.788258
[914]	training's auc: 0.796776	valid_1's auc: 0.788258
[915]	training's auc: 0.796787	valid_1's auc: 0.788262
[916]	training's auc: 0.79679	valid_1's auc: 0.788262
[917]	training's auc: 0.796797	valid_1's auc: 0.788268
[918]	traini

[1049]	training's auc: 0.797396	valid_1's auc: 0.788387
[1050]	training's auc: 0.797397	valid_1's auc: 0.788387
[1051]	training's auc: 0.797401	valid_1's auc: 0.788389
[1052]	training's auc: 0.797403	valid_1's auc: 0.788389
[1053]	training's auc: 0.797404	valid_1's auc: 0.78839
[1054]	training's auc: 0.797409	valid_1's auc: 0.788394
[1055]	training's auc: 0.797425	valid_1's auc: 0.788394
[1056]	training's auc: 0.797431	valid_1's auc: 0.788394
[1057]	training's auc: 0.797432	valid_1's auc: 0.788393
[1058]	training's auc: 0.797443	valid_1's auc: 0.788396
[1059]	training's auc: 0.797445	valid_1's auc: 0.788396
[1060]	training's auc: 0.797455	valid_1's auc: 0.788398
[1061]	training's auc: 0.797462	valid_1's auc: 0.788398
[1062]	training's auc: 0.797469	valid_1's auc: 0.788398
[1063]	training's auc: 0.797471	valid_1's auc: 0.788398
[1064]	training's auc: 0.797474	valid_1's auc: 0.788397
[1065]	training's auc: 0.797482	valid_1's auc: 0.788397
[1066]	training's auc: 0.797491	valid_1's auc: 0.

[1196]	training's auc: 0.797957	valid_1's auc: 0.788476
[1197]	training's auc: 0.797958	valid_1's auc: 0.788476
[1198]	training's auc: 0.79796	valid_1's auc: 0.788476
[1199]	training's auc: 0.797969	valid_1's auc: 0.788479
[1200]	training's auc: 0.797971	valid_1's auc: 0.788479
[1201]	training's auc: 0.797972	valid_1's auc: 0.788479
[1202]	training's auc: 0.797977	valid_1's auc: 0.788482
[1203]	training's auc: 0.797984	valid_1's auc: 0.788488
[1204]	training's auc: 0.797991	valid_1's auc: 0.788496
[1205]	training's auc: 0.797994	valid_1's auc: 0.788496
[1206]	training's auc: 0.798001	valid_1's auc: 0.788495
[1207]	training's auc: 0.798002	valid_1's auc: 0.788495
[1208]	training's auc: 0.798004	valid_1's auc: 0.788495
[1209]	training's auc: 0.798011	valid_1's auc: 0.788501
[1210]	training's auc: 0.798012	valid_1's auc: 0.788501
[1211]	training's auc: 0.798014	valid_1's auc: 0.788502
[1212]	training's auc: 0.798026	valid_1's auc: 0.788503
[1213]	training's auc: 0.798031	valid_1's auc: 0.

[1343]	training's auc: 0.798435	valid_1's auc: 0.78858
[1344]	training's auc: 0.798436	valid_1's auc: 0.78858
[1345]	training's auc: 0.79844	valid_1's auc: 0.788582
[1346]	training's auc: 0.798442	valid_1's auc: 0.788582
[1347]	training's auc: 0.798443	valid_1's auc: 0.788582
[1348]	training's auc: 0.798445	valid_1's auc: 0.788584
[1349]	training's auc: 0.79845	valid_1's auc: 0.788586
[1350]	training's auc: 0.798452	valid_1's auc: 0.788586
[1351]	training's auc: 0.798454	valid_1's auc: 0.788586
[1352]	training's auc: 0.798455	valid_1's auc: 0.788586
[1353]	training's auc: 0.798457	valid_1's auc: 0.788586
[1354]	training's auc: 0.798458	valid_1's auc: 0.788587
[1355]	training's auc: 0.79846	valid_1's auc: 0.788587
[1356]	training's auc: 0.798462	valid_1's auc: 0.788587
[1357]	training's auc: 0.798463	valid_1's auc: 0.788587
[1358]	training's auc: 0.798465	valid_1's auc: 0.788587
[1359]	training's auc: 0.798466	valid_1's auc: 0.788587
[1360]	training's auc: 0.798468	valid_1's auc: 0.7885

In [73]:
tmp = pd.DataFrame(zip(clfs[0].feature_importance(importance_type='gain'), features), columns=['Value', 'Feature'])
tmp.sort_values('Value', ascending=False)

,Value,Feature
10,5.472814e+07,content_explation_true_mean
6,3.361779e+07,content_correctness_std
2,2.671660e+07,content_id
20,2.001365e+07,user_part_correctness
13,1.896011e+07,part_bundle_id
4,1.879862e+07,user_correctness
16,8.168947e+06,content_lagtime
8,7.834301e+06,content_uncorrect_count
15,7.579026e+06,lagtime_in_lecture
11,5.485326e+06,attempt_no


# save

In [74]:
print(type(clfs[0]))

<class 'lightgbm.basic.Booster'>


In [75]:
 from sklearn.externals import joblib
# save model
joblib.dump(clfs[0], 'lgb2.pkl')
# load model
model = joblib.load('lgb2.pkl')


/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [76]:
print(type(model))

<class 'lightgbm.basic.Booster'>
